## Пайплайн на данный момент

- Скачивание зипов фоток с извлечением эмбедов (CLIP - ViT-H-14), также не рабочая часть с извлечением эмбедов из текста (тоже CLIP) и добавление сразу фичей (стоит подумать когда лучше извлекать эмбеды с помощью CLIP из текста)
- Обработка текста с заменой слов и смайлов. Далее эмбеды с помощью берт и CLIP (возможно клип не тут и мб обучить берт)
- Обработка датафрейма: добавление эмбеддинги картинок и текста 128, фичей совпадение категорий и фичи размера фоток (переделать)
- Обучение catboost Kfold = 10

## Зависимости 

Все эксперименты проводились на kaggle.

In [1]:
!pip install open-clip-torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.5 MB/s eta 0:00:00


## Первичная обработка данных

В этом разделе будет приводится пример обработки только на одной части данных, так как в соревновании был выдан внушительный объем данных (6 .parquet файлов и 38 файлов .zip с изображениями).

### Препроцессинг .parquet файлов 

- перевод названий брендов на один язык в title и description
- удалени эмодзи и добавление фичей на основе эмодзи (количество в base и cand, наличие одинаковых в base и cand)
- добавление следующих фичей: price_ratio, imgcount_ratio, base_description_len, cand_description_len, descr_len_ratio, base_title_len, cand_title_len, title_len_ratio, same_subcategory, same_category

In [ ]:
brand_mapping = {
    # ====== Электроника и гаджеты ======
    'айфон': 'iphone',
    'ифон': 'iphone',
    'эпл': 'apple',
    'макбук': 'macbook',
    'айпад': 'ipad',
    'айпод': 'ipod',
    'самсунг': 'samsung',
    'шяоми': 'xiaomi',
    'сяоми': 'xiaomi',
    'редми': 'redmi',
    'по́ко': 'poco',
    'поко': 'poco',
    'хуавей': 'huawei',
    'хонор': 'honor',
    'сони': 'sony',
    'нокиа': 'nokia',
    'моторола': 'motorola',
    'азус': 'asus',
    'асус': 'asus',
    'леново': 'lenovo',
    'делл': 'dell',
    'хп': 'hp',
    'элджи': 'lg',
    'реалми': 'realme',
    'тесла': 'tesla',
    'гугл': 'google',
    'ми': 'mi',
    'инфиникс': 'infinix',
    'тэкно': 'tecno',
    'блекберри': 'blackberry',
    'виво': 'vivo',
    'оппо': 'oppo',

    # ====== Бытовая техника ======
    'бош': 'bosch',
    'борк': 'bork',
    'сименс': 'siemens',
    'электролюкс': 'electrolux',
    'индезит': 'indesit',
    'индесит': 'indesit',
    'аристон': 'ariston',
    'полара': 'polaris',
    'поларис': 'polaris',
    'скарлет': 'scarlett',
    'редмонд': 'redmond',
    'мулинекс': 'moulinex',
    'тефаль': 'tefal',
    'филипс': 'philips',
    'браун': 'braun',
    'витек': 'vitek',
    'канди': 'candy',
    'ардо': 'ardo',
    'ханса': 'hansa',
    'беко': 'beko',
    'вирпул': 'whirlpool',

    # ====== Одежда и обувь ======
    'найк': 'nike',
    'адидас': 'adidas',
    'пума': 'puma',
    'рибок': 'reebok',
    'нью баланс': 'new balance',
    'нью бэланс': 'new balance',
    'нью бэлэнс': 'new balance',
    'нью-баланс': 'new balance',
    'нью-баланс': 'new balance',
    'нью-бэланс': 'new balance',
    'нью-бэлэнс': 'new balance',
    'асикс': 'asics',
    'версаче': 'versace',
    'диор': 'dior',
    'каппа': 'kappa',
    'конверс': 'converse',
    'люи витон': 'louis vuitton',
    'валентино': 'valentino',
    'фила': 'fila',
    'коламбия': 'columbia',
    'норс фейс': 'north face',
    'норс-фейс': 'north face',
    'саламон': 'salomon',
    'патагония': 'patagonia',
    'зара': 'zara',
    'хм': 'h&m',
    'биркенсток': 'birkenstock',
    'кальвин кляйн': 'calvin klein',
    'гап': 'gap',
    'левис': 'levis',
    'дольче габбана': 'dolce&gabbana',
    'гучи': 'gucci',
    'прада': 'prada',
    'баленсиага': 'balenciaga',
    'томми хилфигер': 'tommy hilfiger',
    'лакост': 'lacoste',
    'дезл': 'desigual',
    'десигуаль': 'desigual',
    'массмо датти': 'massimo dutti',
    'массимо дутти': 'massimo dutti',
    'мэнгo': 'mango',
    'скетчерс': 'skechers',
    'экко': 'ecco',
    'ральф лорен': 'ralph lauren',
    'тьюми': 'tumi',
    'миу миу': 'miu miu',
    'шарпей': 'charles & keith',
    'гесс': 'guess',
    'респект': 'respect',
    'касио': 'casio',
    'томми джинс': 'tommy jeans',
    'ванс': 'vans',
    'кроссовер': 'crossover',
    'эсприт': 'esprit',

    # ====== Автомобили и запчасти ======
    'ауди': 'audi',
    'бмв': 'bmw',
    'мерседес': 'mercedes',
    'мерс': 'mercedes',
    'вольво': 'volvo',
    'рено': 'renault',
    'форд': 'ford',
    'тойота': 'toyota',
    'хёндай': 'hyundai',
    'хендай': 'hyundai',
    'киа': 'kia',
    'шкода': 'skoda',
    'мазда': 'mazda',
    'мицубиси': 'mitsubishi',
    'сузуки': 'suzuki',
    'хонда': 'honda',
    'шевроле': 'chevrolet',
    'опель': 'opel',
    'пежо': 'peugeot',
    'ситроен': 'citroen',
    'фольксваген': 'volkswagen',
    'вуксваген': 'volkswagen',
    'лада': 'lada',
    'уаз': 'uaz',
    'лексус': 'lexus',
    'инфинити': 'infiniti',
    'джип': 'jeep',
    'ланд ровер': 'land rover',
    'роллс-ройс': 'rolls-royce',
    'феррари': 'ferrari',
    'ламборджини': 'lamborghini',
    'порше': 'porsche',
    'маклерен': 'mclaren',
    'бугатти': 'bugatti',
    'мазерати': 'maserati',
    'альфа romeo': 'alfa romeo',
    'фиат': 'fiat',
    'додж': 'dodge',
    'крайслер': 'chrysler',
    'тесла': 'tesla',
    'ниссан': 'nissan',
    'субару': 'subaru',
    'дэу': 'daewoo',
    'дэу матиз': 'daewoo matiz',
    'чери': 'chery',
    'джак': 'jac',
    'джей эй си': 'jac',
    'джейси': 'jacie',

    # ====== Детские товары ======
    'лего': 'lego',
    'фишер прайс': 'fisher-price',
    'чикко': 'chicco',
    'хуго': 'hugo',
    'барби': 'barbie',
    'нёрф': 'nerf',
    'хот вилс': 'hot wheels',
    'пегас': 'pegas',
    'сильван': 'sylvanian families',
    'микки маус': 'mickey mouse',
    'бэби бон': 'baby born',
    'кукла лол': 'l.o.l.',
    'пинки пай': 'pinkie pie',
    'мега блокс': 'megabloks',
    'сильван фэмили': 'sylvanian families',
    'сильваниан фэмили': 'sylvanian families',
    'сильваниан': 'sylvanian families',
    'сильвани фэмилис': 'sylvanian families',
    'сильваниан фэмилис': 'sylvanian families',
    'лейго': 'lego',  # частые опечатки
    'легго': 'lego',

    # ====== Косметика и парфюмерия ======
    'лорэаль': 'loreal',
    'лореаль': 'loreal',
    'мак': 'mac',
    'ньюкс': 'nyx',
    'мери кей': 'mary kay',
    'авон': 'avon',
    'эйвон': 'avon',
    'орфлейм': 'oriflame',
    'фаберлик': 'faberlic',
    'эйвон': 'avon',
    'шанэль': 'chanel',
    'диор': 'dior',
    'гивэнчи': 'givenchy',
    'живанши': 'givenchy',
    'ив сен лоран': 'yves saint laurent',
    'клиник': 'clinique',
    'кларанс': 'clarins',
    'эсте лаудер': 'estee lauder',
    'натура сиберика': 'natura siberica',
    'либридерм': 'librederm',

    # ====== Продукты питания ======
    'нескафе': 'nescafe',
    'милки вей': 'milky way',
    'сникерс': 'snickers',
    'баунти': 'bounty',
    'марс': 'mars',
    'кэдбери': 'cadbury',
    'рафаэлло': 'raffaello',
    'ферреро роше': 'ferrero rocher',
    'нестле': 'nestle',
    'данон': 'danone',
    'активия': 'activia',
    'милка': 'milka',
    'альпен гольд': 'alpen gold',
    'тоффи': 'toffifee',
    'липтон': 'lipton',
    'якобс': 'jacobs',
    'бондюэль': 'bonduelle',

    'икеа': 'ikea',
    'икея': 'ikea',
    'джум': 'jum',
    'озон': 'ozon',
    'вилдберриз': 'wildberries',
    'валберис': 'wildberries',
    'вайлдберис': 'wildberries',
    'вайлберис': 'wildberries',
    'валбериз': 'wildberries',
    'вайлдбериз': 'wildberries',
    'вайлбериз': 'wildberries',
    'вб': 'wildberries',
    'яндекс маркет': 'yandex market',
    'макита': 'makita',
    'макитта': 'makita',
    'деволт': 'dewalt',
    'девольт': 'dewalt',
    'хитачи': 'hitachi',
    'интерскол': 'interskol',
    'интерсколл': 'interskol',
    'метабо': 'metabo',
    'енкор': 'enkor',
    'енкорр': 'enkor',
    'эйнхелл': 'einhell',
    'крафтул': 'kraftool',
    'крафттул': 'kraftool',

    # ====== Стройматериалы ======
    'кнауф': 'knauf',
    'кнауфф': 'knauf',
    'гипрок': 'gyproc',
    'сен-гобен': 'saint-gobain',
    'ротбанд': 'rotband',
    'церезит': 'ceresit',
    'церезитт': 'ceresit',
    'тиккурила': 'tikkurila',
    'тикурила': 'tikkurila',
    'дюлюкс': 'dulux',
    'дюлюкс': 'dulux',
    'капарол': 'caparol',
    'хенкель': 'henkel',
    'хенкел': 'henkel',
    'лигалюкс': 'ligalux',

    # ====== Сантехника ======
    'грое': 'grohe',
    'грохэ': 'grohe',
    'грох': 'grohe',
    'рока': 'roca',
    'рокка': 'roca',

    # ====== Электрооборудование ======
    'легранд': 'legrand',
    'легран': 'legrand',
    'шнайдер': 'schneider electric',
    'шнайдер электрик': 'schneider electric',

    'йиск': 'jysk',
    'юск': 'jysk',
    'ашлей': 'ashley',
    'ашли': 'ashley',
    'хемнэс': 'hemnes',
    'микке': 'micke',
    'билли': 'billy',
    'поэнг': 'poäng',

    # ====== Премиум ======
    'херман миллер': 'herman miller',
    'херманмиллер': 'herman miller',
    'ээмс': 'ems',
    'витра': 'vitra',
    'кнутсенсен': 'knutsenсен',
    'роув': 'roche bobois',

    # =====️= Российские бренды ======
    'шатура': 'shatura',
    'штур': 'shtur',
    'бюрократ': 'byurokrat',
    'мебельлюкс': 'mebelux',
    'столплит': 'stolplit',
    'фо́рмекс': 'formex',

    # ====== Онлайн-ритейл ======
    'вейфер': 'wayfair',
    'вэйфэр': 'wayfair',
    'аллиэкспресс мебель': 'aliexpress',
    'озон мебель': 'ozon',

    # ====== Офисная мебель ======
    'стилкейс': 'steelcase',
    'стилкейс': 'steelcase',
    'хаворт': 'haworth',
    'ховарт': 'haworth',
    'офисмен': 'officeman',
    'командор': 'komandor',

    # ====== Кухни ======
    'ноблия': 'nobilia',
    'нобилия': 'nobilia',
    'кухнимания': 'kuhnimania',
    'фаберлик': 'faberlic',
    'крайслер кухни': 'kreisler',

    # =====️= Матрасы ======
    'орматек': 'ormatek',
    'асонс': 'askona',
    'аскона': 'askona',
    'лонтракс': 'lontrax',
    'динекс': 'dinex',

    'пакс': 'PAX',
    'паксш': 'PAX',
    'пэкc': 'PAX',
    'мальм': 'MALM',
    'мальмь': 'MALM',
    'хемнэс': 'HEMNES',
    'хемнес': 'HEMNES',
    'бруса': 'BRUSALI',
    'брусали': 'BRUSALI',
    'квартал': 'KVARTAL',
    'квартэл': 'KVARTAL',

    # ====== Диваны и кресла ======
    'фекторп': 'EKTORP',
    'экторп': 'EKTORP',
    'поэнг': 'POÄNG',
    'поанг': 'POÄNG',
    'фрихетен': 'FRIHETEN',
    'фрихетэн': 'FRIHETEN',
    'клиппан': 'KLIPPAN',
    'клиппанн': 'KLIPPAN',
    'клипан': 'KLIPPAN',

    # ====== Стеллажи и хранение ======
    'каллакс': 'KALLAX',
    'калакс': 'KALLAX',
    'коллакс': 'KALLAX',
    'эльвари': 'ELVARLI',
    'элварли': 'ELVARLI',
    'бурхулт': 'BURHULT',
    'буриульт': 'BURHULT',

    'сектион': 'SEKTION',
    'секцион': 'SEKTION',
    'семсвик': 'SEMSVIK',

    # ====== Детская мебель ======
    'стува': 'STUVA',
    'стувва': 'STUVA',
    'сундвик': 'SUNDVIK',
    'сундвикк': 'SUNDVIK',
    'сниглар': 'SNIGLAR',
    'сниглер': 'SNIGLAR',

    # ====== Столы ======
    'микке': 'MICKE',
    'мике': 'MICKE',

    # ====== Кровати ======
    'несда': 'NESNA',
    'несна': 'NESNA',
    'хёуге': 'HAUGA',
    'хауга': 'HAUGA',
    'слакт': 'SLÄKT',
    'слект': 'SLÄKT',
}

In [ ]:
import pandas as pd
import re
import numpy as np
from tqdm import tqdm
from collections import defaultdict
import matplotlib.pyplot as plt
import gc
import seaborn as sns

def process_brand_replacement(df, brand_mapping, text_columns, batch_size=5000):
    """
    Замена названий брендов в текстовых колонках DataFrame с оптимизацией памяти

    Параметры:
    df - исходный DataFrame
    brand_mapping - словарь {оригинальное_название: замена}
    text_columns - список обрабатываемых колонок (по умолчанию базовые колонки)
    batch_size - размер пакета для обработки
    """
    for col in text_columns:
        df[col] = df[col].astype('string')

    brands = [re.escape(brand) for brand in brand_mapping]
    combined_pattern = re.compile(r'\b(' + '|'.join(brands) + r')\b', flags=re.IGNORECASE)
    replacement_counts = defaultdict(int)

    def replace_brand(match):
        """Функция замены с подсчетом"""
        brand = match.group(0).lower()
        replacement = brand_mapping.get(brand, brand)
        replacement_counts[brand] += 1
        return replacement

    for col in text_columns:
        print(f"Обработка колонки: {col}")
        with tqdm(total=len(df), desc=col, unit="row") as pbar:
            for start in range(0, len(df), batch_size):
                end = min(start + batch_size, len(df))

                batch = df[col].iloc[start:end].apply(
                    lambda x: combined_pattern.sub(replace_brand, x)
                    if isinstance(x, str) else x)

                df.loc[start:end-1, col] = batch
                del batch
                gc.collect()
                pbar.update(end - start)
    print("\nСтатистика замен:")
    for brand, count in sorted(replacement_counts.items(), key=lambda x: x[1], reverse=True)[:20]:
        print(f"{brand}: {count} замен")
    print(f"\nВсего замен: {sum(replacement_counts.values())}")

    return df
    
def full_preprocess(df1):
  df1['price_ratio'] = (df1['base_price'] + 1e-5)/ (df1['cand_price'] + 1e-5)

  df1['imgcount_ratio'] = (df1['base_count_images'] + 1e-5)/ (df1['cand_count_images'] + 1e-5)

  df1['base_description_len'] = df1['base_description'].apply(len)
  df1['cand_description_len'] = df1['cand_description'].apply(len)
  df1['descr_len_ratio'] = (df1['cand_description_len'] + 1e-5)/ (df1['base_description_len'] + 1e-5)

  df1['base_title_len'] = df1['base_title'].apply(len)
  df1['cand_title_len'] = df1['cand_title'].apply(len)
  df1['title_len_ratio'] = (df1['cand_title_len'] + 1e-5)/ (df1['base_title_len'] + 1e-5)

  df1['same_subcategory'] = (df1['base_subcategory_name'] == df1['cand_subcategory_name']).apply(int)
  df1['same_category'] = (df1['base_category_name'] == df1['cand_category_name']).apply(int)

  for col in ['param1', 'param2']:
    df1[f'same_{col}'] = (
        df1[f'base_{col}'].astype(str) ==
        df1[f'cand_{col}'].astype(str)
    ).astype('bool')

  emoji_pattern = re.compile(
    r'[\U0001F600-\U0001F64F'  # Эмодзи-эмотиконы
    r'\U0001F300-\U0001F5FF'   # Символы и пиктограммы
    r'\U0001F680-\U0001F6FF'   # Транспорт и карты
    r'\U0001F900-\U0001F9FF'   # Дополнительные символы
    r'\U0001FA70-\U0001FAFF'   # Символы расширения-A
    r'\U00002702-\U000027B0'   # Дингбаты
    r'\U000024C2-\U0001F251]',
    flags=re.UNICODE)
  df1['base_emojis'] = df1['base_description'].str.findall(emoji_pattern)
  df1['cand_emojis'] = df1['cand_description'].str.findall(emoji_pattern)
  df1['base_has_emojis'] = df1['base_emojis'].str.len() > 0
  df1['cand_has_emojis'] = df1['cand_emojis'].str.len() > 0
  common_emojis_list = []
  for idx, row in df1.iterrows():
      if row['base_has_emojis'] and row['cand_has_emojis']:
          common = set(row['base_emojis']) & set(row['cand_emojis'])
          common_emojis_list.append(len(common) > 0)
      else:
          common_emojis_list.append(False)
  df1['has_common_emojis'] = common_emojis_list
  df1['base_emoji_count'] = df1['base_emojis'].str.len()
  df1['cand_emoji_count'] = df1['cand_emojis'].str.len()
  df1['base_description'] = df1['base_description'].str.replace(emoji_pattern, '', regex=True)
  df1['cand_description'] = df1['cand_description'].str.replace(emoji_pattern, '', regex=True)
  df1.drop(columns=['base_emojis', 'cand_emojis'], inplace=True)

  return df1


In [ ]:
cols_to_check = ['base_title', 'cand_title', 'base_description', 'cand_description']
df1 = pd.read_parquet('https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/Avitotechcomp2025/data_competition_2/train_part_0004.snappy.parquet')
df_processed1 = process_brand_replacement(df1, brand_mapping, cols_to_check)
train1_fullprepr = full_preprocess(df_processed1)
train1_fullprepr.to_parquet("full_processed_data4.parquet", index=False)

### ЭМБЕДДИНГИ

- Эмбеддинги изоражений извлекались с помощью `CLIP ViT-H-14`
- Эмбеддинги текста извлекались из title и description с помощью `CLIP`, также из title, description и json_params извлекались эмбеддинги с помощью берта `cointegrated/rubert-tiny2`.

Извлечения эмбеддингов из изображений и текста с помощью CLIP, для ускорения использовалась параллелизация на 4 картах L4. Полученные эмбеддинги сохранялись в файлы .npz (trainx_image_embeddings.npz и imagex_image_embeddings.npz)

In [2]:
import os
import zipfile
import requests
import torch
import open_clip
import numpy as np
from PIL import Image
from tqdm import tqdm
import pandas as pd
from io import BytesIO
import gc
import shutil
import torch.nn as nn

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 512 if DEVICE == "cuda" else 32
TORCH_DTYPE = torch.float16 if DEVICE == "cuda" else torch.float32
TEMP_DIR = "/tmp/avito_ml_challenge"
os.makedirs(TEMP_DIR, exist_ok=True)

class ImageEncoderWrapper(nn.Module):
    def __init__(self, clip_model):
        super().__init__()
        self.clip_model = clip_model
        
    def forward(self, images):
        return self.clip_model.encode_image(images)

class TextEncoderWrapper(nn.Module):
    def __init__(self, clip_model):
        super().__init__()
        self.clip_model = clip_model
        self.tokenizer = open_clip.get_tokenizer('ViT-H-14')
        
    def forward(self, texts):
        tokens = self.tokenizer(texts).to(DEVICE)
        return self.clip_model.encode_text(tokens)

def initialize_models():
    model, _, preprocess = open_clip.create_model_and_transforms(
        'ViT-H-14',
        pretrained='laion2b-s32b-b79k',
        device=DEVICE,
        precision='fp16' if TORCH_DTYPE == torch.float16 else 'fp32'
    )
    
    image_encoder = ImageEncoderWrapper(model)
    text_encoder = TextEncoderWrapper(model)
    
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs!")
        image_encoder = nn.DataParallel(image_encoder)
        text_encoder = nn.DataParallel(text_encoder)
    
    image_encoder.to(DEVICE)
    text_encoder.to(DEVICE)
    
    return image_encoder, text_encoder, preprocess

image_encoder, text_encoder, preprocess = initialize_models()

def download_file(url, save_path):
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    
    with tqdm(total=total_size, unit='B', unit_scale=True, 
             desc=f"Downloading {os.path.basename(save_path)}") as pbar:
        with open(save_path, 'wb') as f:
            for chunk in response.iter_content(1024*1024):
                f.write(chunk)
                pbar.update(len(chunk))

def process_images(part_num=1):
    base_url = "https://storage.yandexcloud.net/avitotechmlchallenge2025-2/train_title_images_decr/train_images_part_0004-chunk_"
    urls = [f"{base_url}{i:04d}.zip" for i in range(1, 7)]
    
    image_embeddings = {}
    image_sizes = {}

    for url in tqdm(urls, desc="Processing image chunks"):
        zip_name = os.path.basename(url)
        zip_path = os.path.join(TEMP_DIR, zip_name)
        extract_dir = os.path.join(TEMP_DIR, f"images_part_{part_num:04d}")
        
        download_file(url, zip_path)
        
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        
        image_paths = []
        for root, _, files in os.walk(extract_dir):
            image_paths.extend([os.path.join(root, f) for f in files if f.endswith(('.jpg', '.png', '.jpeg'))])

        for i in range(0, len(image_paths), BATCH_SIZE):
            batch_paths = image_paths[i:i+BATCH_SIZE]
            batch_images = []
            valid_paths = []
            
            for path in batch_paths:
                try:
                    img = Image.open(path).convert("RGB")
                    batch_images.append(preprocess(img))
                    valid_paths.append(path)
                except:
                    continue
            
            if not batch_images:
                continue
                
            images_tensor = torch.stack(batch_images).to(DEVICE)
            
            with torch.no_grad(), torch.cuda.amp.autocast():
                embeddings = image_encoder(images_tensor)
                embeddings /= embeddings.norm(dim=-1, keepdim=True)
            
            for path, emb in zip(valid_paths, embeddings.cpu().numpy().astype(np.float16)):
                filename = os.path.basename(path)
                image_embeddings[filename] = emb
                with Image.open(path) as img:
                    image_sizes[filename] = img.size
            
            del images_tensor, embeddings
            torch.cuda.empty_cache()
        
        shutil.rmtree(extract_dir)
        os.remove(zip_path)

    np.savez_compressed(
        "train4_image_embeddings.npz",
        embeddings=image_embeddings,
        sizes=image_sizes
    )

def process_texts(df):
    text_fields = [
        'base_description', 'cand_description',
        'base_title', 'cand_title'
    ]

    text_data = {}
    for field in text_fields:
        unique_texts = df[field].unique().tolist()
        text_data[field] = {text: None for text in unique_texts}

    for field in text_fields:
        texts = list(text_data[field].keys())
        
        for i in tqdm(range(0, len(texts), BATCH_SIZE), desc=f"Processing {field}"):
            batch_texts = texts[i:i+BATCH_SIZE]
            
            with torch.no_grad(), torch.cuda.amp.autocast():
                embeddings = text_encoder(batch_texts)
                embeddings /= embeddings.norm(dim=-1, keepdim=True)
            
            for text, emb in zip(batch_texts, embeddings.cpu().numpy().astype(np.float16)):
                text_data[field][text] = emb

    np.savez_compressed(
        "train4_text_embeddings.npz",
        **{k: dict(v) for k, v in text_data.items()}
    )

def process_data():
    url = "https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/Avitotechcomp2025/data_competition_2/train_part_0004.snappy.parquet"
    df = pd.read_parquet(url)
    process_images()
    process_texts(df)
    
if __name__ == "__main__":
    try:
        process_data()
    finally:
        if os.path.exists(TEMP_DIR):
            shutil.rmtree(TEMP_DIR)
        torch.cuda.empty_cache()

open_clip_model.safetensors:   0%|          | 0.00/3.94G [00:00<?, ?B/s]

Using 4 GPUs!


Processing image chunks:   0%|          | 0/6 [00:00<?, ?it/s]
<ipython-input-2-d49f300b6575>:116: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast():
Processing base_description:   0%|          | 0/255 [00:00<?, ?it/s]<ipython-input-2-d49f300b6575>:158: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast():
Processing cand_title: 100%|██████████| 468/468 [12:28<00:00,  1.60s/it]


Извлечение эмебддингов текста с помощью `cointegrated/rubert-tiny2`, также используется параллелизация. Также как одно из улучшений расматривается замена `cointegrated/rubert-tiny2` на FRIDA или дообученный `sbert` (`bert` обученный с помощью metric learning).
Тут данные сохраняются в формате `.npy` (`train_partx_base_bert_description_embs.npy`)

In [3]:
import torch
import json
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModel
from torch.nn import DataParallel
from torch import nn
import gc
from sklearn.metrics.pairwise import cosine_similarity

MODEL_NAME = 'cointegrated/rubert-tiny2'
FRIDA_MODEL_NAME = 'ai-forever/FRIDA'
BATCH_SIZE = 256
MAX_LENGTH = 512
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
TORCH_DTYPE = torch.float16 if 'cuda' in DEVICE else torch.float32

class EmbedderWrapper:
    def __init__(self):
        self.model = None
        self.tokenizer = None

    def get_embeddings(self, texts_list):
        raise NotImplementedError

class BertEmbedder(EmbedderWrapper):
    def __init__(self, model_name):
        super().__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        
        if torch.cuda.device_count() > 1:
            self.model = DataParallel(self.model)
        
        self.model.to(DEVICE)
        self.model.eval()

    def get_embeddings(self, texts_list):
        embeddings = []
        for i in range(0, len(texts_list), BATCH_SIZE):
            batch = texts_list[i:i+BATCH_SIZE]
            
            inputs = self.tokenizer(
                batch,
                padding=True,
                truncation=True,
                max_length=MAX_LENGTH,
                return_tensors="pt"
            ).to(DEVICE)
            
            with torch.no_grad():
                outputs = self.model(**inputs)
                batch_emb = outputs.last_hidden_state.mean(dim=1)
                embeddings.append(batch_emb.cpu().float().numpy())
            
            del inputs, outputs, batch_emb
            torch.cuda.empty_cache()
        
        return np.concatenate(embeddings)

class FridaEmbedder(EmbedderWrapper):
    def __init__(self, model_name):
        super().__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        
        if torch.cuda.device_count() > 1:
            self.model = DataParallel(self.model)
        
        self.model.to(DEVICE)
        self.model.eval()

    def get_embeddings(self, texts_list):
        embeddings = []
        for i in range(0, len(texts_list), BATCH_SIZE):
            batch = texts_list[i:i+BATCH_SIZE]

            inputs = self.tokenizer(
                batch,
                padding=True,
                truncation=True,
                max_length=MAX_LENGTH,
                return_tensors="pt"
            ).to(DEVICE)

            decoder_inputs = self.tokenizer(
                batch,
                padding=True,
                truncation=True,
                max_length=MAX_LENGTH,
                return_tensors="pt"
            ).to(DEVICE)
            
            with torch.no_grad():

                outputs = self.model(
                    input_ids=inputs.input_ids,
                    attention_mask=inputs.attention_mask,
                    decoder_input_ids=decoder_inputs.input_ids,
                    decoder_attention_mask=decoder_inputs.attention_mask
                )
                # Use the encoder's last hidden state for embeddings
                batch_emb = self.mean_pooling(outputs.encoder_last_hidden_state, inputs['attention_mask'])
                embeddings.append(batch_emb.cpu().float().numpy())
            
            del inputs, decoder_inputs, outputs, batch_emb
            torch.cuda.empty_cache()
        
        return np.concatenate(embeddings)

    def mean_pooling(self, token_embeddings, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return sum_embeddings / sum_mask

def process_json_params(json_str):
    try:
        params = json.loads(json_str)
        return ' '.join(f"{k}:{v}" for k, v in params.items())
    except:
        return ''

def calculate_and_save_similarity(base_embs, cand_embs, df, col_name):
    sim = cosine_similarity(base_embs, cand_embs).diagonal()
    df[col_name] = sim
    del base_embs, cand_embs
    gc.collect()
    return df

def main():

    df = pd.read_parquet(
        "/kaggle/input/part2-preprop-v/full_processed_data4.parquet"
    )

    bert_embedder = BertEmbedder(MODEL_NAME)
    frida_embedder = FridaEmbedder(FRIDA_MODEL_NAME)

    text_types = ['title', 'description', 'json_params']
    frida_text_types = ['title', 'description', 'json_params']

    # Обработка BERT
    for t in text_types:
        print(f"Processing BERT {t}...")

        base_texts = [str(x).strip() for x in df[f'base_{t}']]
        base_embs = bert_embedder.get_embeddings(base_texts)
        np.save(f'train_part1_base_bert_{t}_embs.npy', base_embs)
        del base_texts, base_embs
        gc.collect()

        cand_texts = [str(x).strip() for x in df[f'cand_{t}']]
        cand_embs = bert_embedder.get_embeddings(cand_texts)
        np.save(f'train_part1_cand_bert_{t}_embs.npy', cand_embs)
        del cand_texts, cand_embs
        gc.collect()

    # Обработка FRIDA
    # for t in frida_text_types:
    #     print(f"Processing FRIDA {t}...")
        
    #     # Base processing
    #     base_texts = [str(x).strip() for x in df[f'base_{t}']]
    #     base_embs = frida_embedder.get_embeddings(base_texts)
    #     np.save(f'train_part3_base_frida_{t}_embs.npy', base_embs)
        
    #     # Candidate processing
    #     cand_texts = [str(x).strip() for x in df[f'cand_{t}']]
    #     cand_embs = frida_embedder.get_embeddings(cand_texts)
    #     np.save(f'train_part3_cand_frida_{t}_embs.npy', cand_embs)
        
    #     df = calculate_and_save_similarity(base_embs, cand_embs, df, f'cosine_frida_{t}')
    #     del base_texts, cand_texts, base_embs, cand_embs
    #     gc.collect()


if __name__ == "__main__":
    main()

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/2.70M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.68M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/5.59M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

Processing BERT title...
Processing BERT description...
Processing BERT json_params...
Processing completed!


## Основная обработка данных  

В этом разделе приведены все ячейки для создания данных, необходимых для обучения и предсказания. Для воспроизведения результатов можно запускать ноутбук, начиная с этого раздела, так как все полученные ранее данные хранятся в Kaggle Datasets.  

К данному моменту для каждой части данных имеются следующие файлы:  
- `trainx_image_embeddings.npz`  
- `trainx_text_embeddings.npz`  
- `train_partx_base_bert_description_embs.npy`  
- `train_partx_base_bert_json_params_embs.npy`  
- `train_partx_base_bert_title_embs.npy`  
- `train_partx_cand_bert_description_embs.npy`  
- `train_partx_cand_bert_json_params_embs.npy`  
- `train_partx_cand_bert_title_embs.npy`  
- `full_processed_datax.parquet` (где x = 1–4 для train, 1–2 для test; также для теста отличается `testx_fullprepr.parquet`).  

В результате выполнения каждой ячейки получается один `.parquet`-файл для соответствующей части.  

Для эмбеддингов рассчитываются следующие косинусные расстояния:  
- между изображениями `band` и `cand`;  
- между изображением и текстом (`title` и `description`) — CLIP;  
- между `title` `cand` и `base` (BERT);  
- между `description` `cand` и `base` (BERT).  

Эти косинусные расстояния добавляются в качестве фичей. Также создаются фичи, связанные с разрешением фотографий. Кроме того, добавляются сами эмбеддинги, но с уменьшением размерности до 128 с помощью PCA.  


In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.decomposition import PCA
from tqdm import tqdm
import os
import gc
from math import gcd
from sklearn.metrics.pairwise import cosine_similarity

def optimize_memory(df):
    """Оптимизация типов данных для уменьшения памяти"""
    for col in df.columns:
        dtype = df[col].dtype
        
        if dtype == 'float64':
            df[col] = df[col].astype('float32')
        elif dtype == 'int64':
            if df[col].min() >= 0:
                df[col] = pd.to_numeric(df[col], downcast='unsigned')
            else:
                df[col] = pd.to_numeric(df[col], downcast='integer')
        elif dtype == 'object':
            df[col] = df[col].astype('category')
    return df

def cosine_similarity_metric(embedding1: torch.Tensor, embedding2: torch.Tensor) -> float:
    """
    Вычисление косинусного сходства между двумя эмбеддингами
    
    Параметры:
        embedding1 (torch.Tensor): Первый эмбеддинг [D]
        embedding2 (torch.Tensor): Второй эмбеддинг [D]
        
    Возвращает:
        float: Косинусное сходство в диапазоне [-1, 1]
    """
    embedding1 = embedding1 / embedding1.norm()
    embedding2 = embedding2 / embedding2.norm()
    return torch.dot(embedding1, embedding2).item()

def batch_cosine_similarity(batch1: np.ndarray, batch2: np.ndarray) -> np.ndarray:
    """
    Векторизованная версия для обработки батчей эмбеддингов
    """
    # Нормализуем векторы
    batch1_norm = batch1 / np.linalg.norm(batch1, axis=1, keepdims=True)
    batch2_norm = batch2 / np.linalg.norm(batch2, axis=1, keepdims=True)
    
    # Вычисляем попарные косинусные сходства
    return np.einsum('ij,ij->i', batch1_norm, batch2_norm)

def load_test_embeddings(emb_template, parts, t):
    """Загрузка и объединение эмбеддингов для тестовых частей"""
    embs = []
    for part in parts:
        emb_path = emb_template.format(part=part, t=t)
        if os.path.exists(emb_path):
            embs.append(np.load(emb_path, mmap_mode='r'))
    return np.concatenate(embs) if embs else np.array([])

def simplify_aspect_ratio(width, height):
    """Преобразование размера в упрощенное соотношение сторон"""
    if width == 0 or height == 0:
        return "0x0"
    divisor = gcd(width, height)
    if divisor == 0:
        return "0x0"
    return f"{width//divisor}x{height//divisor}"

def main():
    df1 = pd.read_parquet("/kaggle/input/test-avito/test1_fullprepr.parquet")
    df2 = pd.read_parquet("/kaggle/input/test-avito/test2_fullprepr.parquet")
    df = pd.concat([df1, df2], axis=0)
    test_parts = [1, 2]
    
    # Оптимизация памяти
    df = optimize_memory(df)
    
    # 1. Загрузка CLIP эмбеддингов
    clip_text_embs = np.load("/kaggle/input/test-avito/test_text_embeddings.npz", allow_pickle=True)
    base_title_clip = clip_text_embs['base_title'].item()
    cand_title_clip = clip_text_embs['cand_title'].item()
    base_desc_clip = clip_text_embs['base_description'].item()
    cand_desc_clip = clip_text_embs['cand_description'].item()
    
    # 2. Обработка текстовых эмбеддингов
    text_types = ['title', 'description', 'params']
    pca = PCA(n_components=128)

    for t in text_types:
        # Загрузка объединенных эмбеддингов
        base_embs = load_test_embeddings(
            '/kaggle/input/test-avito/test{part}_base_{t}_embs.npy',
            test_parts, t
        )
        
        cand_embs = load_test_embeddings(
            '/kaggle/input/test-avito/test{part}_cand_{t}_embs.npy',
            test_parts, t
        )

        # Расчет косинусного сходства
        chunk_size = 10000
        cosine_sims = np.empty(len(df), dtype='float32')
        
        for i in tqdm(range(0, len(df), chunk_size), desc=f"Cosine {t}"):
            end = min(i+chunk_size, len(df))
            b = base_embs[i:end].astype('float32')
            c = cand_embs[i:end].astype('float32')
            cosine_sims[i:end] = batch_cosine_similarity(b, c)
        
        df[f'text_cosine_sim_{t}'] = cosine_sims.astype('float32')

        # Применение PCA
        combined = np.vstack([base_embs, cand_embs])
        pca.fit(combined)
        
        base_pca = np.empty((len(df), 128), dtype='float32')
        cand_pca = np.empty((len(df), 128), dtype='float32')
        
        for i in tqdm(range(0, len(df), chunk_size), desc=f"PCA {t}"):
            end = min(i+chunk_size, len(df))
            base_pca[i:end] = pca.transform(base_embs[i:end].astype('float32'))
            cand_pca[i:end] = pca.transform(cand_embs[i:end].astype('float32'))

        # Создание DataFrame с колонками
        for i in range(128):
            if t == 'params':
                t_0 = 'json_params'
                df[f'base_{t_0}_emb_{i}'] = base_pca[:, i].astype('float32')
                df[f'cand_{t_0}_emb_{i}'] = cand_pca[:, i].astype('float32')
            else:
                df[f'base_{t}_emb_{i}'] = base_pca[:, i].astype('float32')
                df[f'cand_{t}_emb_{i}'] = cand_pca[:, i].astype('float32')
            
        del base_embs, cand_embs, base_pca, cand_pca, combined, b, c, cosine_sims
        gc.collect()
    
    # 3. Обработка изображений
    image_data = np.load("/kaggle/input/test-avito/test_embeddings.npz")
    all_image_embs = image_data['embeddings']
    all_sizes = image_data['sizes']
    image_filenames = image_data['filenames']

    # Создаем маппинги
    image_filename_map = {os.path.splitext(fn)[0]: idx for idx, fn in enumerate(image_filenames)}
    filename_to_size = {os.path.splitext(fn)[0]: size for fn, size in zip(image_filenames, all_sizes)}

    # Функция для получения размеров изображений
    def get_image_size_features(row, prefix):
        fn = os.path.splitext(row[f'{prefix}_title_image'])[0]
        width, height = filename_to_size.get(fn, (0, 0))
        aspect_ratio = simplify_aspect_ratio(width, height)
        return pd.Series({
            f'{prefix}_width': width,
            f'{prefix}_height': height,
            f'{prefix}_aspect_ratio': aspect_ratio
        })

    # Добавляем фичи для base и cand изображений
    for prefix in ['base', 'cand']:
        size_features = df.apply(lambda row: get_image_size_features(row, prefix), axis=1)
        df[[f'{prefix}_width', f'{prefix}_height', f'{prefix}_aspect_ratio']] = size_features
        del size_features
        gc.collect()
        df[f'{prefix}_aspect_ratio'] = df[f'{prefix}_aspect_ratio'].astype('category')
    
    # Добавляем фичу для сравнения соотношений сторон
    df['same_aspect_ratio'] = (df['base_aspect_ratio'].astype(str) == df['cand_aspect_ratio'].astype(str)).astype('bool')

    # Расчет косинусного сходства для изображений
    chunk_size = 10000
    image_cosine_sim = np.empty(len(df), dtype='float32')
    
    for i in tqdm(range(0, len(df), chunk_size), desc="Image Cosine"):
        end = min(i+chunk_size, len(df))
        batch = df.iloc[i:end]
        
        base_indices = batch['base_title_image'].apply(
            lambda x: image_filename_map.get(os.path.splitext(x)[0], -1))
        cand_indices = batch['cand_title_image'].apply(
            lambda x: image_filename_map.get(os.path.splitext(x)[0], -1))
        
        valid_mask = (base_indices >= 0) & (cand_indices >= 0)
        
        if valid_mask.any():
            base_batch = all_image_embs[base_indices[valid_mask]].astype('float32')
            cand_batch = all_image_embs[cand_indices[valid_mask]].astype('float32')
            image_cosine_sim[i:end][valid_mask] = batch_cosine_similarity(base_batch, cand_batch)
        
        # Для невалидных пар устанавливаем нейтральное значение 0
        image_cosine_sim[i:end][~valid_mask] = 0.0
    
    df['image_cosine_sim'] = image_cosine_sim.astype('float32')

    # Применение PCA для изображений
    pca = PCA(n_components=128)
    image_embs_pca = pca.fit_transform(all_image_embs).astype('float32')
    
    # Векторизованное получение эмбеддингов
    def get_image_embeddings(images):
        indices = images.apply(
            lambda x: image_filename_map.get(os.path.splitext(x)[0], -1))
        valid_mask = indices >= 0
        embeddings = np.full((len(images), 128), fill_value=0.0, dtype='float32')
        embeddings[valid_mask] = image_embs_pca[indices[valid_mask]]
        return embeddings
    
    base_image_embs = get_image_embeddings(df['base_title_image'])
    cand_image_embs = get_image_embeddings(df['cand_title_image'])
    
    # Создание колонок для эмбеддингов изображений
    for i in range(128):
        df[f'base_img_emb_{i}'] = base_image_embs[:, i]
        df[f'cand_img_emb_{i}'] = cand_image_embs[:, i]
    
    # 4. Косинусное сходство CLIP текстов с изображениями
    def get_clip_image_cosine_similarity(row, text_field, clip_dict, image_field):
        text = row[text_field]
        image_fn = os.path.splitext(row[image_field])[0]  # Убираем .jpg если есть
        
        text_emb = clip_dict.get(text, None)
        image_idx = image_filename_map.get(image_fn, -1)
        
        if text_emb is not None and image_idx >= 0:
            image_emb = all_image_embs[image_idx]
            return cosine_similarity_metric(
                torch.from_numpy(text_emb).float(),
                torch.from_numpy(image_emb).float()
            )
        return 0.0
    
    clip_fields = [
        ('base_title', base_title_clip),
        ('cand_title', cand_title_clip),
        ('base_description', base_desc_clip),
        ('cand_description', cand_desc_clip)
    ]
    
    for text_field, clip_dict in clip_fields:
        for img_field in ['base_title_image', 'cand_title_image']:
            col_name = f'{text_field}_clip_cosine_sim_{img_field.split("_")[0]}_img'
            df[col_name] = df.apply(
                lambda row: get_clip_image_cosine_similarity(row, text_field, clip_dict, img_field), 
                axis=1
            ).astype('float32')
    
    # 5. PCA для CLIP текстов
    all_clip_embs = []
    for d in [base_title_clip, cand_title_clip, base_desc_clip, cand_desc_clip]:
        all_clip_embs.extend([emb for emb in d.values() if emb is not None and isinstance(emb, np.ndarray)])
    
    # Проверяем, что все эмбеддинги имеют одинаковую размерность
    emb_dims = set(emb.shape[0] for emb in all_clip_embs)
    if len(emb_dims) > 1:
        print(f"Предупреждение: обнаружены эмбеддинги разной размерности: {emb_dims}")
        min_dim = min(emb_dims)
        all_clip_embs = [emb[:min_dim] for emb in all_clip_embs]
    
    all_clip_embs_array = np.vstack(all_clip_embs).astype('float32')
    pca_clip = PCA(n_components=128).fit(all_clip_embs_array)
    
    def get_clip_pca_embeddings(texts, clip_dict):
        """Безопасное получение CLIP эмбеддингов"""
        default_emb = np.zeros(pca_clip.n_features_in_, dtype='float32')
        embs = []
        for text in texts:
            emb = clip_dict.get(text, None)
            if emb is not None and isinstance(emb, np.ndarray):
                if emb.shape[0] > pca_clip.n_features_in_:
                    emb = emb[:pca_clip.n_features_in_]
                elif emb.shape[0] < pca_clip.n_features_in_:
                    temp = np.zeros(pca_clip.n_features_in_, dtype='float32')
                    temp[:emb.shape[0]] = emb
                    emb = temp
                embs.append(emb)
            else:
                embs.append(default_emb)
        return pca_clip.transform(np.vstack(embs)).astype('float32')
    
    for text_field, clip_dict in clip_fields:
        pca_embs = get_clip_pca_embeddings(df[text_field], clip_dict)
        for i in range(128):
            df[f'{text_field}_clip_emb_{i}'] = pca_embs[:, i]
    
    # 6. Финализация
    text_cols = ['base_json_params', 'cand_json_params']
    for col in text_cols:
        df[f'{col}_len'] = df[col].str.len().fillna(0).astype('int16')
    
    cols_to_drop = text_cols + [
        'base_title', 'cand_title', 'base_description', 'cand_description', 
        'base_title_image', 'cand_title_image'
    ]
    df.drop(columns=[col for col in cols_to_drop if col in df.columns], inplace=True)
    
    df = optimize_memory(df)
    df.to_parquet('processed_test_data_embs_PCA.parquet', index=False)
    print(f"Финальный размер датафрейма: {df.shape}")

if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.decomposition import PCA
from tqdm import tqdm
import os
import gc
from math import gcd
from sklearn.metrics.pairwise import cosine_similarity

def optimize_memory(df):
    """Оптимизация типов данных для уменьшения памяти"""
    for col in df.columns:
        dtype = df[col].dtype
        
        if dtype == 'float64':
            df[col] = df[col].astype('float32')
        elif dtype == 'int64':
            if df[col].min() >= 0:
                df[col] = pd.to_numeric(df[col], downcast='unsigned')
            else:
                df[col] = pd.to_numeric(df[col], downcast='integer')
        elif dtype == 'object':
            df[col] = df[col].astype('category')
    return df

def cosine_similarity_metric(embedding1: torch.Tensor, embedding2: torch.Tensor) -> float:
    """
    Вычисление косинусного сходства между двумя эмбеддингами
    
    Параметры:
        embedding1 (torch.Tensor): Первый эмбеддинг [D]
        embedding2 (torch.Tensor): Второй эмбеддинг [D]
        
    Возвращает:
        float: Косинусное сходство в диапазоне [-1, 1]
    """
    embedding1 = embedding1 / embedding1.norm()
    embedding2 = embedding2 / embedding2.norm()
    return torch.dot(embedding1, embedding2).item()

def batch_cosine_similarity(batch1: np.ndarray, batch2: np.ndarray) -> np.ndarray:
    """
    Векторизованная версия для обработки батчей эмбеддингов
    """
    # Нормализуем векторы
    batch1_norm = batch1 / np.linalg.norm(batch1, axis=1, keepdims=True)
    batch2_norm = batch2 / np.linalg.norm(batch2, axis=1, keepdims=True)
    
    # Вычисляем попарные косинусные сходства
    return np.einsum('ij,ij->i', batch1_norm, batch2_norm)

def load_test_embeddings(emb_template, parts, t):
    """Загрузка и объединение эмбеддингов для тестовых частей"""
    embs = []
    for part in parts:
        emb_path = emb_template.format(t=t)
        if os.path.exists(emb_path):
            embs.append(np.load(emb_path, mmap_mode='r'))
    return np.concatenate(embs) if embs else np.array([])

def simplify_aspect_ratio(width, height):
    """Преобразование размера в упрощенное соотношение сторон"""
    if width == 0 or height == 0:
        return "0x0"
    divisor = gcd(width, height)
    if divisor == 0:
        return "0x0"
    return f"{width//divisor}x{height//divisor}"

def main():
    df = pd.read_parquet("/kaggle/input/part2-preprop-v/full_processed_data1.parquet")
    test_parts = [1, 2]
    df = optimize_memory(df)
    
    # 1. Загрузка CLIP эмбеддингов
    clip_text_embs = np.load("/kaggle/input/part-1-avito/train_text_embeddings.npz", allow_pickle=True)
    base_title_clip = clip_text_embs['base_title'].item()
    cand_title_clip = clip_text_embs['cand_title'].item()
    base_desc_clip = clip_text_embs['base_description'].item()
    cand_desc_clip = clip_text_embs['cand_description'].item()
    
    # 2. Обработка BERT эмбеддингов
    text_types = ['title', 'description', 'json_params']
    pca = PCA(n_components=128)

    for t in text_types:
        base_embs = load_test_embeddings('/kaggle/input/part-1-avito/train_part1_base_bert_{t}_embs.npy', test_parts, t)
        cand_embs = load_test_embeddings('/kaggle/input/part-1-avito/train_part1_cand_bert_{t}_embs.npy', test_parts, t)

        # Расчет косинусного сходства
        chunk_size = 10000
        similarities = np.empty(len(df), dtype='float32')
        
        for i in tqdm(range(0, len(df), chunk_size), desc=f"Cosine Similarity {t}"):
            end = min(i+chunk_size, len(df))
            b = base_embs[i:end].astype('float32')
            c = cand_embs[i:end].astype('float32')
            similarities[i:end] = batch_cosine_similarity(b, c)
        
        df[f'text_cosine_sim_{t}'] = similarities.astype('float32')

        # Применение PCA
        combined = np.vstack([base_embs, cand_embs])
        pca.fit(combined)
        
        base_pca = np.empty((len(df), 128), dtype='float32')
        cand_pca = np.empty((len(df), 128), dtype='float32')
        
        for i in tqdm(range(0, len(df), chunk_size), desc=f"PCA {t}"):
            end = min(i+chunk_size, len(df))
            base_pca[i:end] = pca.transform(base_embs[i:end].astype('float32'))
            cand_pca[i:end] = pca.transform(cand_embs[i:end].astype('float32'))

        for i in range(128):
            col_name = 'json_params' if t == 'params' else t
            df[f'base_{col_name}_emb_{i}'] = base_pca[:, i]
            df[f'cand_{col_name}_emb_{i}'] = cand_pca[:, i]
            
        del base_embs, cand_embs, base_pca, cand_pca, combined, similarities
        gc.collect()
    
    # 3. Обработка изображений
    image_data = np.load("/kaggle/input/part-1-avito/train_image_embeddings.npz", allow_pickle=True)
    image_embeddings = image_data['embeddings'].item()
    image_sizes = image_data['sizes'].item()
    
    # Размеры изображений
    def get_image_size_features(row, prefix):
        filename = row[f'{prefix}_title_image'] + '.jpg'
        size = image_sizes.get(filename, (0, 0))
        return pd.Series({
            f'{prefix}_width': size[0],
            f'{prefix}_height': size[1],
            f'{prefix}_aspect_ratio': simplify_aspect_ratio(*size)
        })
    
    for prefix in ['base', 'cand']:
        size_features = df.apply(lambda row: get_image_size_features(row, prefix), axis=1)
        df[[f'{prefix}_width', f'{prefix}_height', f'{prefix}_aspect_ratio']] = size_features
        df[f'{prefix}_aspect_ratio'] = df[f'{prefix}_aspect_ratio'].astype('category')
        del size_features
        gc.collect()
    
    df['same_aspect_ratio'] = (df['base_aspect_ratio'].astype(str) == df['cand_aspect_ratio'].astype(str)).astype('bool')

    # Косинусное сходство изображений
    chunk_size = 10000
    image_cosine_sim = np.zeros(len(df), dtype='float32')
    
    for i in tqdm(range(0, len(df), chunk_size), desc="Image Cosine Similarity"):
        end = min(i+chunk_size, len(df))
        batch = df.iloc[i:end]
        
        base_fns = batch['base_title_image'].astype(str) + '.jpg'
        cand_fns = batch['cand_title_image'].astype(str) + '.jpg'
            
        valid_mask = base_fns.isin(image_embeddings) & cand_fns.isin(image_embeddings)
        
        if valid_mask.any():
            base_batch = np.array([image_embeddings[fn] for fn in base_fns[valid_mask]], dtype='float32')
            cand_batch = np.array([image_embeddings[fn] for fn in cand_fns[valid_mask]], dtype='float32')
            image_cosine_sim[i:end][valid_mask.values] = batch_cosine_similarity(base_batch, cand_batch)
    
    df['image_cosine_sim'] = image_cosine_sim

    # PCA для изображений
    all_image_embs = np.vstack([emb.reshape(1, -1) if emb.ndim == 1 else emb for emb in image_embeddings.values()])
    pca_img = PCA(n_components=128).fit(all_image_embs.astype('float32'))
    filename_to_pca = {fn: pca_img.transform(emb.reshape(1, -1))[0] for fn, emb in image_embeddings.items()}
    
    def get_image_pca_embeddings(images):
        filenames = images.astype(str) + '.jpg'
        valid_mask = filenames.isin(filename_to_pca)
        embs = np.zeros((len(images), 128), dtype='float32')
        embs[valid_mask] = np.array([filename_to_pca[fn] for fn in filenames[valid_mask]])
        return embs
    
    base_img_pca = get_image_pca_embeddings(df['base_title_image'])
    cand_img_pca = get_image_pca_embeddings(df['cand_title_image'])
    
    for i in range(128):
        df[f'base_img_emb_{i}'] = base_img_pca[:, i]
        df[f'cand_img_emb_{i}'] = cand_img_pca[:, i]
    
    del all_image_embs, filename_to_pca, base_img_pca, cand_img_pca
    gc.collect()
    
    # 4. Косинусное сходство CLIP текстов с изображениями
    def get_clip_image_cosine_similarity(row, text_field, clip_dict, image_field):
        text = row[text_field]
        image_fn = row[image_field] + '.jpg'
        
        text_emb = clip_dict.get(text, None)
        image_emb = image_embeddings.get(image_fn, None)
        
        if text_emb is not None and image_emb is not None:
            return cosine_similarity_metric(
                torch.from_numpy(text_emb).float(),
                torch.from_numpy(image_emb).float()
            )
        return 0.0
    
    clip_fields = [
        ('base_title', base_title_clip),
        ('cand_title', cand_title_clip),
        ('base_description', base_desc_clip),
        ('cand_description', cand_desc_clip)
    ]
    
    for text_field, clip_dict in clip_fields:
        for img_field in ['base_title_image', 'cand_title_image']:
            col_name = f'{text_field}_clip_cosine_sim_{img_field.split("_")[0]}_img'
            df[col_name] = df.apply(
                lambda row: get_clip_image_cosine_similarity(row, text_field, clip_dict, img_field), 
                axis=1
            ).astype('float32')
    
    # 5. PCA для CLIP текстов
    all_clip_embs = []
    for d in [base_title_clip, cand_title_clip, base_desc_clip, cand_desc_clip]:
        all_clip_embs.extend([emb for emb in d.values() if emb is not None and isinstance(emb, np.ndarray)])
    
    # Проверяем, что все эмбеддинги имеют одинаковую размерность
    emb_dims = set(emb.shape[0] for emb in all_clip_embs)
    if len(emb_dims) > 1:
        print(f"Предупреждение: обнаружены эмбеддинги разной размерности: {emb_dims}")
        # Берем минимальную размерность
        min_dim = min(emb_dims)
        all_clip_embs = [emb[:min_dim] for emb in all_clip_embs]
    
    all_clip_embs_array = np.vstack(all_clip_embs).astype('float32')
    pca_clip = PCA(n_components=128).fit(all_clip_embs_array)
    
    def get_clip_pca_embeddings(texts, clip_dict):
        """Безопасное получение CLIP эмбеддингов с обработкой отсутствующих значений"""
        default_emb = np.zeros(pca_clip.n_features_in_, dtype='float32')
        embs = []
        for text in texts:
            emb = clip_dict.get(text, None)
            if emb is not None and isinstance(emb, np.ndarray):
                # Обрезаем до нужной размерности если необходимо
                if emb.shape[0] > pca_clip.n_features_in_:
                    emb = emb[:pca_clip.n_features_in_]
                elif emb.shape[0] < pca_clip.n_features_in_:
                    temp = np.zeros(pca_clip.n_features_in_, dtype='float32')
                    temp[:emb.shape[0]] = emb
                    emb = temp
                embs.append(emb)
            else:
                embs.append(default_emb)
        return pca_clip.transform(np.vstack(embs)).astype('float32')
    
    for text_field, clip_dict in clip_fields:
        pca_embs = get_clip_pca_embeddings(df[text_field], clip_dict)
        for i in range(128):
            df[f'{text_field}_clip_emb_{i}'] = pca_embs[:, i]
    
    # 6. Финализация
    text_cols = ['base_json_params', 'cand_json_params']
    for col in text_cols:
        df[f'{col}_len'] = df[col].str.len().fillna(0).astype('int16')
    
    cols_to_drop = text_cols + [
        'base_title', 'cand_title', 'base_description', 'cand_description', 
        'base_title_image', 'cand_title_image'
    ]
    df.drop(columns=[col for col in cols_to_drop if col in df.columns], inplace=True)
    
    df = optimize_memory(df)
    df.to_parquet('processed_data1_embs_PCA.parquet', index=False)
    print(f"Финальный размер датафрейма: {df.shape}")

if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from tqdm import tqdm
import os
import json
import gc
from math import gcd

def optimize_memory(df):
    """Оптимизация типов данных для уменьшения памяти"""
    for col in df.columns:
        dtype = df[col].dtype
        
        if dtype == 'float64':
            df[col] = df[col].astype('float32')

        
        elif dtype == 'int64':
            if df[col].min() >= 0:
                df[col] = pd.to_numeric(df[col], downcast='unsigned')
            else:
                df[col] = pd.to_numeric(df[col], downcast='integer')
                
        elif dtype == 'object':
            df[col] = df[col].astype('category')
    
    return df

def calculate_cosine_distance(batch1, batch2):
    """Вычисление косинусного расстояния для батчей векторов"""
    norm = np.linalg.norm(batch1, axis=1) * np.linalg.norm(batch2, axis=1)
    cosine_sim = np.einsum('ij,ij->i', batch1, batch2) / np.where(norm == 0, 1e-6, norm)
    return cosine_sim

def load_test_embeddings(emb_template, parts, t):
    """Загрузка и объединение эмбеддингов для тестовых частей"""
    embs = []
    for part in parts:
        emb_path = emb_template.format(t=t)
        if os.path.exists(emb_path):
            embs.append(np.load(emb_path, mmap_mode='r'))
    return np.concatenate(embs) if embs else np.array([])

def simplify_aspect_ratio(width, height):
    """Преобразование размера в упрощенное соотношение сторон"""
    if width == 0 or height == 0:
        return "0x0"
    
    # Находим наибольший общий делитель
    divisor = gcd(width, height)
    if divisor == 0:
        return "0x0"
    
    simplified_w = width // divisor
    simplified_h = height // divisor
    

    return f"{simplified_w}x{simplified_h}"

def main():
    df = pd.read_parquet("/kaggle/input/part2-preprop-v/full_processed_data2.parquet")
    test_parts = [1, 2]
    
    # Оптимизация памяти
    df = optimize_memory(df)
    
    # 1. Загрузка CLIP эмбеддингов из нового файла
    clip_embs = np.load("/kaggle/input/part2-preprop-v/train2_text_embeddings.npz", allow_pickle=True)
    base_title_clip = clip_embs['base_title'].item()
    cand_title_clip = clip_embs['cand_title'].item()
    base_desc_clip = clip_embs['base_description'].item()
    cand_desc_clip = clip_embs['cand_description'].item()
    
    # 2. Обработка текстовых эмбеддингов
    text_types = ['title', 'description', 'json_params']
    pca = PCA(n_components=128)

    for t in text_types:
        # Загрузка объединенных эмбеддингов
        base_embs = load_test_embeddings(
            '/kaggle/input/part2-replaced/train_part2_base_bert_{t}_embs.npy',
            test_parts, t
        )
        
        cand_embs = load_test_embeddings(
            '/kaggle/input/part2-replaced/train_part2_cand_bert_{t}_embs.npy',
            test_parts, t
        )

        # Расчет косинусного расстояния
        chunk_size = 10000
        cosine_dists = np.empty(len(df), dtype='float32')
        
        for i in tqdm(range(0, len(df), chunk_size), desc=f"Cosine {t}"):
            end = min(i+chunk_size, len(df))
            b = base_embs[i:end].astype('float32')
            c = cand_embs[i:end].astype('float32')
            cosine_dists[i:end] = calculate_cosine_distance(b, c)
        
        df[f'text_cosine_{t}'] = cosine_dists.astype('float32')

        # Применение PCA
        combined = np.vstack([base_embs, cand_embs])
        pca.fit(combined)
        
        base_pca = np.empty((len(df), 128), dtype='float32')
        cand_pca = np.empty((len(df), 128), dtype='float32')
        
        for i in tqdm(range(0, len(df), chunk_size), desc=f"PCA {t}"):
            end = min(i+chunk_size, len(df))
            base_pca[i:end] = pca.transform(base_embs[i:end].astype('float32'))
            cand_pca[i:end] = pca.transform(cand_embs[i:end].astype('float32'))

        # Создание DataFrame с колонками
        for i in range(128):
            if t == 'params':
                t_0 = 'json_params'
                df[f'base_{t_0}_emb_{i}'] = base_pca[:, i].astype('float32')
                df[f'cand_{t_0}_emb_{i}'] = cand_pca[:, i].astype('float32')
            else:
                df[f'base_{t}_emb_{i}'] = base_pca[:, i].astype('float32')
                df[f'cand_{t}_emb_{i}'] = cand_pca[:, i].astype('float32')
            
        del base_embs, cand_embs, base_pca, cand_pca, combined, b, c, cosine_dists
        gc.collect()
    
    # 3. Обработка изображений
    image_data = np.load("/kaggle/input/part2-replaced/part_0002_embeddings.npz")
    all_image_embs = image_data['embeddings']
    all_sizes = image_data['sizes']
    image_filenames = image_data['filenames']

    # Создаем маппинги
    image_filename_map = {os.path.splitext(fn)[0]: idx for idx, fn in enumerate(image_filenames)}
    filename_to_size = {os.path.splitext(fn)[0]: size for fn, size in zip(image_filenames, all_sizes)}

    # Функция для получения размеров изображений
    def get_image_size_features(row, prefix):
        fn = os.path.splitext(row[f'{prefix}_title_image'])[0]
        width, height = filename_to_size.get(fn, (0, 0))
        aspect_ratio = simplify_aspect_ratio(width, height)
        return pd.Series({
            f'{prefix}_width': width,
            f'{prefix}_height': height,
            f'{prefix}_aspect_ratio': aspect_ratio
        })

    # Добавляем фичи для base и cand изображений
    for prefix in ['base', 'cand']:
        size_features = df.apply(lambda row: get_image_size_features(row, prefix), axis=1)
        df[[f'{prefix}_width', f'{prefix}_height', f'{prefix}_aspect_ratio']] = size_features
        del size_features
        gc.collect()
        # Преобразуем в категориальные типы
        df[f'{prefix}_aspect_ratio'] = df[f'{prefix}_aspect_ratio'].astype('category')
        

    # Добавляем фичу для сравнения соотношений сторон
    df['same_aspect_ratio'] = (df['base_aspect_ratio'].astype(str) == df['cand_aspect_ratio'].astype(str)).astype('bool')

    # Расчет косинусного расстояния для изображений
    chunk_size = 10000
    image_cosine = np.empty(len(df), dtype='float32')
    
    for i in tqdm(range(0, len(df), chunk_size), desc="Image Cosine"):
        end = min(i+chunk_size, len(df))
        batch = df.iloc[i:end]
        
        base_indices = batch['base_title_image'].apply(
            lambda x: image_filename_map.get(os.path.splitext(x)[0], -1))
        cand_indices = batch['cand_title_image'].apply(
            lambda x: image_filename_map.get(os.path.splitext(x)[0], -1))
        
        valid_mask = (base_indices >= 0) & (cand_indices >= 0)
        
        base_batch = all_image_embs[base_indices[valid_mask]].astype('float32')
        cand_batch = all_image_embs[cand_indices[valid_mask]].astype('float32')
        distances = calculate_cosine_distance(base_batch, cand_batch)
        
        image_cosine[i:end][valid_mask] = distances
        image_cosine[i:end][~valid_mask] = 1.0
    
    df['image_cosine'] = image_cosine.astype('float32')

    # Применение PCA для изображений
    pca = PCA(n_components=128)
    image_embs_pca = pca.fit_transform(all_image_embs).astype('float32')
    
    # Векторизованное получение эмбеддингов
    def get_image_embeddings(images):
        indices = images.apply(
            lambda x: image_filename_map.get(os.path.splitext(x)[0], -1))
        valid_mask = indices >= 0
        embeddings = np.full((len(images), 128), fill_value=0.0, dtype='float32')
        embeddings[valid_mask] = image_embs_pca[indices[valid_mask]]
        return embeddings
    
    base_image_embs = get_image_embeddings(df['base_title_image'])
    cand_image_embs = get_image_embeddings(df['cand_title_image'])
    
    # Создание колонок для эмбеддингов изображений
    for i in range(128):
        df[f'base_img_emb_{i}'] = base_image_embs[:, i]
        df[f'cand_img_emb_{i}'] = cand_image_embs[:, i]
    
    # 4. Добавляем косинусное сходство между CLIP текстами и изображениями
    def get_clip_image_cosine_similarity(row, text_field, clip_dict, image_field):
        text = str(row[text_field])
        image_fn = os.path.splitext(row[image_field])[0]
        
        text_emb = clip_dict.get(text, None)
        image_idx = image_filename_map.get(image_fn, -1)
        image_emb = all_image_embs[image_idx] if image_idx >= 0 else None
        
        if text_emb is not None and image_emb is not None:
            text_emb = text_emb.astype('float32')
            image_emb = image_emb.astype('float32')
            norm = np.linalg.norm(text_emb) * np.linalg.norm(image_emb)
            if norm == 0:
                return 0.0
            return np.dot(text_emb, image_emb) / norm
        return 0.0
    
    clip_fields = [
        ('base_title', base_title_clip),
        ('cand_title', cand_title_clip),
        ('base_description', base_desc_clip),
        ('cand_description', cand_desc_clip)
    ]
    
    for text_field, clip_dict in clip_fields:
        for img_field in ['base_title_image', 'cand_title_image']:
            col_name = f'{text_field}_clip_cosine_sim_{img_field.split("_")[0]}_img'
            df[col_name] = df.apply(
                lambda row: get_clip_image_cosine_similarity(row, text_field, clip_dict, img_field), 
                axis=1
            ).astype('float32')
    
    # 5. PCA для CLIP текстов
    all_clip_embs = []
    for d in [base_title_clip, cand_title_clip, base_desc_clip, cand_desc_clip]:
        all_clip_embs.extend([emb for emb in d.values() if emb is not None and isinstance(emb, np.ndarray)])
    
    # Проверяем, что все эмбеддинги имеют одинаковую размерность
    if all_clip_embs:
        emb_dims = set(emb.shape[0] for emb in all_clip_embs)
        if len(emb_dims) > 1:
            print(f"Предупреждение: обнаружены эмбеддинги разной размерности: {emb_dims}")
            # Берем минимальную размерность
            min_dim = min(emb_dims)
            all_clip_embs = [emb[:min_dim] for emb in all_clip_embs]
        
        all_clip_embs_array = np.vstack(all_clip_embs).astype('float32')
        pca_clip = PCA(n_components=128).fit(all_clip_embs_array)
        
        def get_clip_pca_embeddings(texts, clip_dict):
            """Безопасное получение CLIP эмбеддингов с обработкой отсутствующих значений"""
            default_emb = np.zeros(pca_clip.n_features_in_, dtype='float32')
            embs = []
            for text in texts:
                emb = clip_dict.get(str(text), None)
                if emb is not None and isinstance(emb, np.ndarray):
                    # Обрезаем до нужной размерности если необходимо
                    if emb.shape[0] > pca_clip.n_features_in_:
                        emb = emb[:pca_clip.n_features_in_]
                    elif emb.shape[0] < pca_clip.n_features_in_:
                        temp = np.zeros(pca_clip.n_features_in_, dtype='float32')
                        temp[:emb.shape[0]] = emb
                        emb = temp
                    embs.append(emb)
                else:
                    embs.append(default_emb)
            return pca_clip.transform(np.vstack(embs)).astype('float32')
        
        for text_field, clip_dict in clip_fields:
            pca_embs = get_clip_pca_embeddings(df[text_field], clip_dict)
            for i in range(128):
                df[f'{text_field}_clip_emb_{i}'] = pca_embs[:, i]
    
    # 6. Финализация
    text_cols = ['base_json_params', 'cand_json_params']
    for col in text_cols:
        df[f'{col}_len'] = df[col].str.len().fillna(0).astype('int16')
    
    # Удаляем ненужные колонки
    cols_to_drop = text_cols + [
        'base_title', 'cand_title', 'base_description', 'cand_description', 'base_title_image', 'cand_title_image',
    ]
    df.drop(columns=[col for col in cols_to_drop if col in df.columns], inplace=True)
    
    # Финализация оптимизации
    df = optimize_memory(df)
    df.to_parquet('processed_data2_embs_PCA.parquet', index=False)
    print(f"Финальный размер датафрейма: {df.shape}")

if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.decomposition import PCA
from tqdm import tqdm
import os
import gc
from math import gcd
from sklearn.metrics.pairwise import cosine_similarity

def optimize_memory(df):
    """Оптимизация типов данных для уменьшения памяти"""
    for col in df.columns:
        dtype = df[col].dtype
        
        if dtype == 'float64':
            df[col] = df[col].astype('float32')
        elif dtype == 'int64':
            if df[col].min() >= 0:
                df[col] = pd.to_numeric(df[col], downcast='unsigned')
            else:
                df[col] = pd.to_numeric(df[col], downcast='integer')
        elif dtype == 'object':
            df[col] = df[col].astype('category')
    return df

def cosine_similarity_metric(embedding1: torch.Tensor, embedding2: torch.Tensor) -> float:
    """
    Вычисление косинусного сходства между двумя эмбеддингами
    
    Параметры:
        embedding1 (torch.Tensor): Первый эмбеддинг [D]
        embedding2 (torch.Tensor): Второй эмбеддинг [D]
        
    Возвращает:
        float: Косинусное сходство в диапазоне [-1, 1]
    """
    embedding1 = embedding1 / embedding1.norm()
    embedding2 = embedding2 / embedding2.norm()
    return torch.dot(embedding1, embedding2).item()

def batch_cosine_similarity(batch1: np.ndarray, batch2: np.ndarray) -> np.ndarray:
    """
    Векторизованная версия для обработки батчей эмбеддингов
    """
    # Нормализуем векторы
    batch1_norm = batch1 / np.linalg.norm(batch1, axis=1, keepdims=True)
    batch2_norm = batch2 / np.linalg.norm(batch2, axis=1, keepdims=True)
    
    # Вычисляем попарные косинусные сходства
    return np.einsum('ij,ij->i', batch1_norm, batch2_norm)

def load_test_embeddings(emb_template, parts, t):
    """Загрузка и объединение эмбеддингов для тестовых частей"""
    embs = []
    for part in parts:
        emb_path = emb_template.format(t=t)
        if os.path.exists(emb_path):
            embs.append(np.load(emb_path, mmap_mode='r'))
    return np.concatenate(embs) if embs else np.array([])

def simplify_aspect_ratio(width, height):
    """Преобразование размера в упрощенное соотношение сторон"""
    if width == 0 or height == 0:
        return "0x0"
    divisor = gcd(width, height)
    if divisor == 0:
        return "0x0"
    return f"{width//divisor}x{height//divisor}"

def main():
    df = pd.read_parquet("/kaggle/input/part2-preprop-v/full_processed_data3.parquet")
    test_parts = [1, 2]
    df = optimize_memory(df)
    
    # 1. Загрузка CLIP эмбеддингов
    clip_text_embs = np.load("/kaggle/input/part-3/train3_text_embeddings.npz", allow_pickle=True)
    base_title_clip = clip_text_embs['base_title'].item()
    cand_title_clip = clip_text_embs['cand_title'].item()
    base_desc_clip = clip_text_embs['base_description'].item()
    cand_desc_clip = clip_text_embs['cand_description'].item()
    
    # 2. Обработка BERT эмбеддингов
    text_types = ['title', 'description', 'json_params']
    pca = PCA(n_components=128)

    for t in text_types:
        base_embs = load_test_embeddings('/kaggle/input/part-3/train_part3_base_bert_{t}_embs.npy', test_parts, t)
        cand_embs = load_test_embeddings('/kaggle/input/part-3/train_part3_cand_bert_{t}_embs.npy', test_parts, t)

        # Расчет косинусного сходства
        chunk_size = 10000
        similarities = np.empty(len(df), dtype='float32')
        
        for i in tqdm(range(0, len(df), chunk_size), desc=f"Cosine Similarity {t}"):
            end = min(i+chunk_size, len(df))
            b = base_embs[i:end].astype('float32')
            c = cand_embs[i:end].astype('float32')
            similarities[i:end] = batch_cosine_similarity(b, c)
        
        df[f'text_cosine_sim_{t}'] = similarities.astype('float32')

        # Применение PCA
        combined = np.vstack([base_embs, cand_embs])
        pca.fit(combined)
        
        base_pca = np.empty((len(df), 128), dtype='float32')
        cand_pca = np.empty((len(df), 128), dtype='float32')
        
        for i in tqdm(range(0, len(df), chunk_size), desc=f"PCA {t}"):
            end = min(i+chunk_size, len(df))
            base_pca[i:end] = pca.transform(base_embs[i:end].astype('float32'))
            cand_pca[i:end] = pca.transform(cand_embs[i:end].astype('float32'))

        for i in range(128):
            col_name = 'json_params' if t == 'params' else t
            df[f'base_{col_name}_emb_{i}'] = base_pca[:, i]
            df[f'cand_{col_name}_emb_{i}'] = cand_pca[:, i]
            
        del base_embs, cand_embs, base_pca, cand_pca, combined, similarities
        gc.collect()
    
    # 3. Обработка изображений
    image_data = np.load("/kaggle/input/part-3/train3_image_embeddings.npz", allow_pickle=True)
    image_embeddings = image_data['embeddings'].item()
    image_sizes = image_data['sizes'].item()
    
    # Размеры изображений
    def get_image_size_features(row, prefix):
        filename = row[f'{prefix}_title_image'] + '.jpg'
        size = image_sizes.get(filename, (0, 0))
        return pd.Series({
            f'{prefix}_width': size[0],
            f'{prefix}_height': size[1],
            f'{prefix}_aspect_ratio': simplify_aspect_ratio(*size)
        })
    
    for prefix in ['base', 'cand']:
        size_features = df.apply(lambda row: get_image_size_features(row, prefix), axis=1)
        df[[f'{prefix}_width', f'{prefix}_height', f'{prefix}_aspect_ratio']] = size_features
        df[f'{prefix}_aspect_ratio'] = df[f'{prefix}_aspect_ratio'].astype('category')
        del size_features
        gc.collect()
    
    df['same_aspect_ratio'] = (df['base_aspect_ratio'].astype(str) == df['cand_aspect_ratio'].astype(str)).astype('bool')

    # Косинусное сходство изображений
    chunk_size = 10000
    image_cosine_sim = np.zeros(len(df), dtype='float32')
    
    for i in tqdm(range(0, len(df), chunk_size), desc="Image Cosine Similarity"):
        end = min(i+chunk_size, len(df))
        batch = df.iloc[i:end]
        
        base_fns = batch['base_title_image'].astype(str) + '.jpg'
        cand_fns = batch['cand_title_image'].astype(str) + '.jpg'
            
        valid_mask = base_fns.isin(image_embeddings) & cand_fns.isin(image_embeddings)
        
        if valid_mask.any():
            base_batch = np.array([image_embeddings[fn] for fn in base_fns[valid_mask]], dtype='float32')
            cand_batch = np.array([image_embeddings[fn] for fn in cand_fns[valid_mask]], dtype='float32')
            image_cosine_sim[i:end][valid_mask.values] = batch_cosine_similarity(base_batch, cand_batch)
    
    df['image_cosine_sim'] = image_cosine_sim

    # PCA для изображений
    all_image_embs = np.vstack([emb.reshape(1, -1) if emb.ndim == 1 else emb for emb in image_embeddings.values()])
    pca_img = PCA(n_components=128).fit(all_image_embs.astype('float32'))
    filename_to_pca = {fn: pca_img.transform(emb.reshape(1, -1))[0] for fn, emb in image_embeddings.items()}
    
    def get_image_pca_embeddings(images):
        filenames = images.astype(str) + '.jpg'
        valid_mask = filenames.isin(filename_to_pca)
        embs = np.zeros((len(images), 128), dtype='float32')
        embs[valid_mask] = np.array([filename_to_pca[fn] for fn in filenames[valid_mask]])
        return embs
    
    base_img_pca = get_image_pca_embeddings(df['base_title_image'])
    cand_img_pca = get_image_pca_embeddings(df['cand_title_image'])
    
    for i in range(128):
        df[f'base_img_emb_{i}'] = base_img_pca[:, i]
        df[f'cand_img_emb_{i}'] = cand_img_pca[:, i]
    
    del all_image_embs, filename_to_pca, base_img_pca, cand_img_pca
    gc.collect()
    
    # 4. Косинусное сходство CLIP текстов с изображениями
    def get_clip_image_cosine_similarity(row, text_field, clip_dict, image_field):
        text = row[text_field]
        image_fn = row[image_field] + '.jpg'
        
        text_emb = clip_dict.get(text, None)
        image_emb = image_embeddings.get(image_fn, None)
        
        if text_emb is not None and image_emb is not None:
            return cosine_similarity_metric(
                torch.from_numpy(text_emb).float(),
                torch.from_numpy(image_emb).float()
            )
        return 0.0
    
    clip_fields = [
        ('base_title', base_title_clip),
        ('cand_title', cand_title_clip),
        ('base_description', base_desc_clip),
        ('cand_description', cand_desc_clip)
    ]
    
    for text_field, clip_dict in clip_fields:
        for img_field in ['base_title_image', 'cand_title_image']:
            col_name = f'{text_field}_clip_cosine_sim_{img_field.split("_")[0]}_img'
            df[col_name] = df.apply(
                lambda row: get_clip_image_cosine_similarity(row, text_field, clip_dict, img_field), 
                axis=1
            ).astype('float32')
    
    # 5. PCA для CLIP текстов
    all_clip_embs = []
    for d in [base_title_clip, cand_title_clip, base_desc_clip, cand_desc_clip]:
        all_clip_embs.extend([emb for emb in d.values() if emb is not None and isinstance(emb, np.ndarray)])
    
    # Проверяем, что все эмбеддинги имеют одинаковую размерность
    emb_dims = set(emb.shape[0] for emb in all_clip_embs)
    if len(emb_dims) > 1:
        print(f"Предупреждение: обнаружены эмбеддинги разной размерности: {emb_dims}")
        # Берем минимальную размерность
        min_dim = min(emb_dims)
        all_clip_embs = [emb[:min_dim] for emb in all_clip_embs]
    
    all_clip_embs_array = np.vstack(all_clip_embs).astype('float32')
    pca_clip = PCA(n_components=128).fit(all_clip_embs_array)
    
    def get_clip_pca_embeddings(texts, clip_dict):
        """Безопасное получение CLIP эмбеддингов с обработкой отсутствующих значений"""
        default_emb = np.zeros(pca_clip.n_features_in_, dtype='float32')
        embs = []
        for text in texts:
            emb = clip_dict.get(text, None)
            if emb is not None and isinstance(emb, np.ndarray):
                # Обрезаем до нужной размерности если необходимо
                if emb.shape[0] > pca_clip.n_features_in_:
                    emb = emb[:pca_clip.n_features_in_]
                elif emb.shape[0] < pca_clip.n_features_in_:
                    temp = np.zeros(pca_clip.n_features_in_, dtype='float32')
                    temp[:emb.shape[0]] = emb
                    emb = temp
                embs.append(emb)
            else:
                embs.append(default_emb)
        return pca_clip.transform(np.vstack(embs)).astype('float32')
    
    for text_field, clip_dict in clip_fields:
        pca_embs = get_clip_pca_embeddings(df[text_field], clip_dict)
        for i in range(128):
            df[f'{text_field}_clip_emb_{i}'] = pca_embs[:, i]
    
    # 6. Финализация
    text_cols = ['base_json_params', 'cand_json_params']
    for col in text_cols:
        df[f'{col}_len'] = df[col].str.len().fillna(0).astype('int16')
    
    cols_to_drop = text_cols + [
        'base_title', 'cand_title', 'base_description', 'cand_description', 
        'base_title_image', 'cand_title_image'
    ]
    df.drop(columns=[col for col in cols_to_drop if col in df.columns], inplace=True)
    
    df = optimize_memory(df)
    df.to_parquet('processed_data3_embs_PCA.parquet', index=False)
    print(f"Финальный размер датафрейма: {df.shape}")

if __name__ == "__main__":
    main()

In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.decomposition import PCA
from tqdm import tqdm
import os
import gc
from math import gcd
from sklearn.metrics.pairwise import cosine_similarity

def optimize_memory(df):
    """Оптимизация типов данных для уменьшения памяти"""
    for col in df.columns:
        dtype = df[col].dtype
        
        if dtype == 'float64':
            df[col] = df[col].astype('float32')
        elif dtype == 'int64':
            if df[col].min() >= 0:
                df[col] = pd.to_numeric(df[col], downcast='unsigned')
            else:
                df[col] = pd.to_numeric(df[col], downcast='integer')
        elif dtype == 'object':
            df[col] = df[col].astype('category')
    return df

def cosine_similarity_metric(embedding1: torch.Tensor, embedding2: torch.Tensor) -> float:
    """
    Вычисление косинусного сходства между двумя эмбеддингами
    
    Параметры:
        embedding1 (torch.Tensor): Первый эмбеддинг [D]
        embedding2 (torch.Tensor): Второй эмбеддинг [D]
        
    Возвращает:
        float: Косинусное сходство в диапазоне [-1, 1]
    """
    embedding1 = embedding1 / embedding1.norm()
    embedding2 = embedding2 / embedding2.norm()
    return torch.dot(embedding1, embedding2).item()

def batch_cosine_similarity(batch1: np.ndarray, batch2: np.ndarray) -> np.ndarray:
    """
    Векторизованная версия для обработки батчей эмбеддингов
    """
    # Нормализуем векторы
    batch1_norm = batch1 / np.linalg.norm(batch1, axis=1, keepdims=True)
    batch2_norm = batch2 / np.linalg.norm(batch2, axis=1, keepdims=True)
    
    # Вычисляем попарные косинусные сходства
    return np.einsum('ij,ij->i', batch1_norm, batch2_norm)

def load_test_embeddings(emb_template, parts, t):
    """Загрузка и объединение эмбеддингов для тестовых частей"""
    embs = []
    for part in parts:
        emb_path = emb_template.format(t=t)
        if os.path.exists(emb_path):
            embs.append(np.load(emb_path, mmap_mode='r'))
    return np.concatenate(embs) if embs else np.array([])

def simplify_aspect_ratio(width, height):
    """Преобразование размера в упрощенное соотношение сторон"""
    if width == 0 or height == 0:
        return "0x0"
    divisor = gcd(width, height)
    if divisor == 0:
        return "0x0"
    return f"{width//divisor}x{height//divisor}"

def main():
    df = pd.read_parquet("/kaggle/input/part2-preprop-v/full_processed_data4.parquet")
    test_parts = [1, 2]
    df = optimize_memory(df)
    
    # 1. Загрузка CLIP эмбеддингов
    clip_text_embs = np.load("/kaggle/input/part-4/train4_text_embeddings.npz", allow_pickle=True)
    base_title_clip = clip_text_embs['base_title'].item()
    cand_title_clip = clip_text_embs['cand_title'].item()
    base_desc_clip = clip_text_embs['base_description'].item()
    cand_desc_clip = clip_text_embs['cand_description'].item()
    
    # 2. Обработка BERT эмбеддингов
    text_types = ['title', 'description', 'json_params']
    pca = PCA(n_components=128)

    for t in text_types:
        base_embs = load_test_embeddings('/kaggle/input/part-4/train_part4_base_bert_{t}_embs.npy', test_parts, t)
        cand_embs = load_test_embeddings('/kaggle/input/part-4/train_part4_cand_bert_{t}_embs.npy', test_parts, t)

        # Расчет косинусного сходства
        chunk_size = 10000
        similarities = np.empty(len(df), dtype='float32')
        
        for i in tqdm(range(0, len(df), chunk_size), desc=f"Cosine Similarity {t}"):
            end = min(i+chunk_size, len(df))
            b = base_embs[i:end].astype('float32')
            c = cand_embs[i:end].astype('float32')
            similarities[i:end] = batch_cosine_similarity(b, c)
        
        df[f'text_cosine_sim_{t}'] = similarities.astype('float32')

        # Применение PCA
        combined = np.vstack([base_embs, cand_embs])
        pca.fit(combined)
        
        base_pca = np.empty((len(df), 128), dtype='float32')
        cand_pca = np.empty((len(df), 128), dtype='float32')
        
        for i in tqdm(range(0, len(df), chunk_size), desc=f"PCA {t}"):
            end = min(i+chunk_size, len(df))
            base_pca[i:end] = pca.transform(base_embs[i:end].astype('float32'))
            cand_pca[i:end] = pca.transform(cand_embs[i:end].astype('float32'))

        for i in range(128):
            col_name = 'json_params' if t == 'params' else t
            df[f'base_{col_name}_emb_{i}'] = base_pca[:, i]
            df[f'cand_{col_name}_emb_{i}'] = cand_pca[:, i]
            
        del base_embs, cand_embs, base_pca, cand_pca, combined, similarities
        gc.collect()
    
    # 3. Обработка изображений
    image_data = np.load("/kaggle/input/part-4/train4_image_embeddings.npz", allow_pickle=True)
    image_embeddings = image_data['embeddings'].item()
    image_sizes = image_data['sizes'].item()
    
    # Размеры изображений
    def get_image_size_features(row, prefix):
        filename = row[f'{prefix}_title_image'] + '.jpg'
        size = image_sizes.get(filename, (0, 0))
        return pd.Series({
            f'{prefix}_width': size[0],
            f'{prefix}_height': size[1],
            f'{prefix}_aspect_ratio': simplify_aspect_ratio(*size)
        })
    
    for prefix in ['base', 'cand']:
        size_features = df.apply(lambda row: get_image_size_features(row, prefix), axis=1)
        df[[f'{prefix}_width', f'{prefix}_height', f'{prefix}_aspect_ratio']] = size_features
        df[f'{prefix}_aspect_ratio'] = df[f'{prefix}_aspect_ratio'].astype('category')
        del size_features
        gc.collect()
    
    df['same_aspect_ratio'] = (df['base_aspect_ratio'].astype(str) == df['cand_aspect_ratio'].astype(str)).astype('bool')

    # Косинусное сходство изображений
    chunk_size = 10000
    image_cosine_sim = np.zeros(len(df), dtype='float32')
    
    for i in tqdm(range(0, len(df), chunk_size), desc="Image Cosine Similarity"):
        end = min(i+chunk_size, len(df))
        batch = df.iloc[i:end]
        
        base_fns = batch['base_title_image'].astype(str) + '.jpg'
        cand_fns = batch['cand_title_image'].astype(str) + '.jpg'
            
        valid_mask = base_fns.isin(image_embeddings) & cand_fns.isin(image_embeddings)
        
        if valid_mask.any():
            base_batch = np.array([image_embeddings[fn] for fn in base_fns[valid_mask]], dtype='float32')
            cand_batch = np.array([image_embeddings[fn] for fn in cand_fns[valid_mask]], dtype='float32')
            image_cosine_sim[i:end][valid_mask.values] = batch_cosine_similarity(base_batch, cand_batch)
    
    df['image_cosine_sim'] = image_cosine_sim

    # PCA для изображений
    all_image_embs = np.vstack([emb.reshape(1, -1) if emb.ndim == 1 else emb for emb in image_embeddings.values()])
    pca_img = PCA(n_components=128).fit(all_image_embs.astype('float32'))
    filename_to_pca = {fn: pca_img.transform(emb.reshape(1, -1))[0] for fn, emb in image_embeddings.items()}
    
    def get_image_pca_embeddings(images):
        filenames = images.astype(str) + '.jpg'
        valid_mask = filenames.isin(filename_to_pca)
        embs = np.zeros((len(images), 128), dtype='float32')
        embs[valid_mask] = np.array([filename_to_pca[fn] for fn in filenames[valid_mask]])
        return embs
    
    base_img_pca = get_image_pca_embeddings(df['base_title_image'])
    cand_img_pca = get_image_pca_embeddings(df['cand_title_image'])
    
    for i in range(128):
        df[f'base_img_emb_{i}'] = base_img_pca[:, i]
        df[f'cand_img_emb_{i}'] = cand_img_pca[:, i]
    
    del all_image_embs, filename_to_pca, base_img_pca, cand_img_pca
    gc.collect()
    
    # 4. Косинусное сходство CLIP текстов с изображениями
    def get_clip_image_cosine_similarity(row, text_field, clip_dict, image_field):
        text = row[text_field]
        image_fn = row[image_field] + '.jpg'
        
        text_emb = clip_dict.get(text, None)
        image_emb = image_embeddings.get(image_fn, None)
        
        if text_emb is not None and image_emb is not None:
            return cosine_similarity_metric(
                torch.from_numpy(text_emb).float(),
                torch.from_numpy(image_emb).float()
            )
        return 0.0
    
    clip_fields = [
        ('base_title', base_title_clip),
        ('cand_title', cand_title_clip),
        ('base_description', base_desc_clip),
        ('cand_description', cand_desc_clip)
    ]
    
    for text_field, clip_dict in clip_fields:
        for img_field in ['base_title_image', 'cand_title_image']:
            col_name = f'{text_field}_clip_cosine_sim_{img_field.split("_")[0]}_img'
            df[col_name] = df.apply(
                lambda row: get_clip_image_cosine_similarity(row, text_field, clip_dict, img_field), 
                axis=1
            ).astype('float32')
    
    # 5. PCA для CLIP текстов
    all_clip_embs = []
    for d in [base_title_clip, cand_title_clip, base_desc_clip, cand_desc_clip]:
        all_clip_embs.extend([emb for emb in d.values() if emb is not None and isinstance(emb, np.ndarray)])
    
    # Проверяем, что все эмбеддинги имеют одинаковую размерность
    emb_dims = set(emb.shape[0] for emb in all_clip_embs)
    if len(emb_dims) > 1:
        print(f"Предупреждение: обнаружены эмбеддинги разной размерности: {emb_dims}")
        # Берем минимальную размерность
        min_dim = min(emb_dims)
        all_clip_embs = [emb[:min_dim] for emb in all_clip_embs]
    
    all_clip_embs_array = np.vstack(all_clip_embs).astype('float32')
    pca_clip = PCA(n_components=128).fit(all_clip_embs_array)
    
    def get_clip_pca_embeddings(texts, clip_dict):
        """Безопасное получение CLIP эмбеддингов с обработкой отсутствующих значений"""
        default_emb = np.zeros(pca_clip.n_features_in_, dtype='float32')
        embs = []
        for text in texts:
            emb = clip_dict.get(text, None)
            if emb is not None and isinstance(emb, np.ndarray):
                # Обрезаем до нужной размерности если необходимо
                if emb.shape[0] > pca_clip.n_features_in_:
                    emb = emb[:pca_clip.n_features_in_]
                elif emb.shape[0] < pca_clip.n_features_in_:
                    temp = np.zeros(pca_clip.n_features_in_, dtype='float32')
                    temp[:emb.shape[0]] = emb
                    emb = temp
                embs.append(emb)
            else:
                embs.append(default_emb)
        return pca_clip.transform(np.vstack(embs)).astype('float32')
    
    for text_field, clip_dict in clip_fields:
        pca_embs = get_clip_pca_embeddings(df[text_field], clip_dict)
        for i in range(128):
            df[f'{text_field}_clip_emb_{i}'] = pca_embs[:, i]
    
    # 6. Финализация
    text_cols = ['base_json_params', 'cand_json_params']
    for col in text_cols:
        df[f'{col}_len'] = df[col].str.len().fillna(0).astype('int16')
    
    cols_to_drop = text_cols + [
        'base_title', 'cand_title', 'base_description', 'cand_description', 
        'base_title_image', 'cand_title_image'
    ]
    df.drop(columns=[col for col in cols_to_drop if col in df.columns], inplace=True)
    
    df = optimize_memory(df)
    df.to_parquet('processed_data4_embs_PCA.parquet', index=False)
    print(f"Финальный размер датафрейма: {df.shape}")

if __name__ == "__main__":
    main()

PCA title: 100%|██████████| 38/38 [00:00<00:00, 57.23it/s]
<ipython-input-1-9e03721659ff>:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'base_{col_name}_emb_{i}'] = base_pca[:, i]
<ipython-input-1-9e03721659ff>:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'cand_{col_name}_emb_{i}'] = cand_pca[:, i]
<ipython-input-1-9e03721659ff>:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at 

Финальный размер датафрейма: (379555, 1593)



## Обучение  

В качестве основной модели использовался **CatBoost** с **StratifiedGroupKFold** для кросс-валидации. Обучение проводилось с подобранными гиперпараметрами, включая:  
```python
'eval_metric': 'PRAUC',  
'early_stopping_rounds': 150  
```  
Это позволило предотвратить переобучение модели и оценить её качество для поставленной задачи.  

В конце обучения выводилась оценка важности признаков 

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import average_precision_score, log_loss
import matplotlib.pyplot as plt
import gc

# 1. Загрузка данных
df1 = pd.read_parquet('processed_data1_embs_PCA.parquet')
df2 = pd.read_parquet('processed_data2_embs_PCA.parquet')
df3 = pd.read_parquet('processed_data3_embs_PCA.parquet')
df4 = pd.read_parquet('processed_data4_embs_PCA.parquet')
df = pd.concat([df1, df2, df3, df4], axis=0)
del df1, df2, df3, df4
gc.collect()
groups = df['group_id']

df.drop(['action_date', 'group_id'], axis=1, inplace=True, errors='ignore')

# 2. Обработка категориальных признаков
cat_columns = [
    'base_category_name', 'cand_category_name',
    'base_subcategory_name', 'cand_subcategory_name',
    'base_param1', 'cand_param1',
    'base_param2', 'cand_param2',
    'same_subcategory', 'same_category',
    'base_aspect_ratio', 'cand_aspect_ratio'   
]

for col in cat_columns:
    if col in df.columns:
        # Обработка категориальных признаков
        df[col] = df[col].astype(str).fillna('missing')
        df[col] = df[col].apply(
            lambda x: f"cat_{x}" if x.replace('.', '', 1).isdigit() else x
        )

# 3. Обработка числовых признаков
for col in df.columns:
    if col in cat_columns:
        continue
    if df[col].dtype == 'object':
        df[col] = df[col].fillna('missing')
    elif pd.api.types.is_numeric_dtype(df[col]):
        df[col] = df[col].fillna(-999).astype('float32')

# 4. Подготовка данных
X = df.drop(columns=['is_double', 'base_item_id', 'cand_item_id'], errors='ignore')
y = df['is_double'].astype(int)

# 5. Параметры модели
params = {
    'iterations': 5000,
    'learning_rate': 0.05,
    'depth': 8,
    'l2_leaf_reg': 15,
    'border_count': 254,  
    'eval_metric': 'PRAUC',
    'early_stopping_rounds': 150,
    'task_type': 'GPU',
    'verbose': 100,
    'loss_function': 'CrossEntropy',
    'subsample': 0.75,
    'bootstrap_type': 'Poisson',
    'min_data_in_leaf': 10,
}

# 6. Кросс-валидация
N_FOLDS = 5
cv = StratifiedGroupKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)
metrics = {'PRAUC': [], 'LogLoss': []}
feature_importances = []
models = []

for fold, (train_idx, val_idx) in enumerate(cv.split(X, y, groups)):
    print(f"\n{'='*40}")
    print(f"Fold {fold+1}/{N_FOLDS}")
    print(f"{'='*40}")
    
    # Разделение данных
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    # Создание Pool
    train_pool = Pool(X_train, y_train, cat_features=cat_columns)
    val_pool = Pool(X_val, y_val, cat_features=cat_columns)
    
    # Обучение
    model = CatBoostClassifier(**params)
    model.fit(train_pool, eval_set=val_pool)
    model.save_model(f'catboost_model_fold_{fold+1}.cbm')
    # Предсказания и метрики
    val_preds = model.predict_proba(val_pool)[:, 1]
    metrics['PRAUC'].append(average_precision_score(y_val, val_preds))
    metrics['LogLoss'].append(log_loss(y_val, val_preds))
    
    # Сохранение результатов
    feature_importances.append(model.get_feature_importance())
    models.append(model)
    
    # Очистка памяти
    del model, train_pool, val_pool
    gc.collect()
    
    print(f"Fold {fold+1} PRAUC: {metrics['PRAUC'][-1]:.4f}")
    print(f"Fold {fold+1} LogLoss: {metrics['LogLoss'][-1]:.4f}")

print("\nCross-Validation Results:")
for metric in metrics:
    mean_score = np.mean(metrics[metric])
    std_score = np.std(metrics[metric])
    print(f"{metric}: {mean_score:.4f} ± {std_score:.4f}")

# 7. Визуализация важности признаков
avg_importance = np.mean(feature_importances, axis=0)
sorted_idx = np.argsort(avg_importance)[-30:]
plt.figure(figsize=(12, 8))
plt.barh(range(30), avg_importance[sorted_idx[-30:]])
plt.yticks(range(30), X.columns[sorted_idx[-30:]])
plt.title("Average Feature Importance Across Folds")
plt.show()

print("\nBest Model Saved!")

## Инференс 

В результате был использован ансамбль из 5 моделей, обученных на предыдущем шаге. С помощью ансамбля выполнялись предсказания на тестовых данных, которые затем сохранялись в файл формата .csv.

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
import joblib

# 1. Загрузка тестовых данных
test_df = pd.read_parquet('/kaggle/working/processed_test_data_embs_PCA.parquet')
test_df['text_cosine_sim_json_params']=test_df['text_cosine_sim_params']
test_df[['text_cosine_title', 'text_cosine_description', 'text_cosine_json_params', 'image_cosine']] = test_df[['text_cosine_sim_title', 'text_cosine_sim_description', 'text_cosine_sim_json_params', 'image_cosine_sim']]

ids_df = test_df[['base_item_id', 'cand_item_id']].copy()

columns_to_drop = ['action_date', 'group_id', 'base_item_id', 'cand_item_id', 'text_cosine_sim_params']
test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns], inplace=True)
test_df = test_df[X.columns]

# 2. Обработка категориальных признаков (как при обучении)
cat_columns = [
    'base_category_name', 'cand_category_name',
    'base_subcategory_name', 'cand_subcategory_name',
    'base_param1', 'cand_param1',
    'base_param2', 'cand_param2',
    'same_subcategory', 'same_category',
    'base_aspect_ratio',  'cand_aspect_ratio'   
]

for col in cat_columns:
    if col in test_df.columns:
        test_df[col] = test_df[col].astype(str)
        test_df[col] = test_df[col].apply(
            lambda x: f"cat_{x}" if x.replace('.', '', 1).isdigit() else x
        )
        test_df[col] = test_df[col].fillna('missing')

# 3. Заполнение пропусков в числовых признаках
for col in test_df.columns:
    if col in cat_columns:
        continue
    if test_df[col].dtype == 'object':
        test_df[col] = test_df[col].fillna('missing')
    elif pd.api.types.is_numeric_dtype(test_df[col]):
        test_df[col] = test_df[col].fillna(-999).astype('float32')

# 4. Загрузка всех 5 моделей
models = []
for fold in range(5):
    model = CatBoostClassifier()
    model.load_model(f'catboost_model_fold_{fold+1}.cbm')
    models.append(model)

# 5. Прогнозирование вероятностей ансамблем
all_preds = []
test_pool = Pool(
    data=test_df,
    cat_features=cat_columns,
    thread_count=-1  
)
for model in models:
    preds = model.predict_proba(
    test_pool,
    task_type='CPU', 
    verbose=100  
)[:, 1] 
    all_preds.append(preds)
    
probabilities = np.mean(all_preds, axis=0)

# 6. Формирование результата
result_df = pd.DataFrame({
    'base_id': ids_df['base_item_id'],
    'cand_id': ids_df['cand_item_id'],
    'probability': probabilities
})

result_df.to_csv('submission_ensemble.csv', index=False)

print("Ансамблевый инференс завершен. Результат сохранен в submission_ensemble.csv")